In [12]:
import requests

import joblib
import os

## Get Models

In [13]:
models = []
files = os.listdir()

for item in files:
    if item.endswith(".pkl"):
        print(f'Loading {item}')
        models.append(joblib.load(item))

Loading Decision Tree Regressor.pkl
Loading Random Forest Regressor.pkl


## Get Stock Info

In [ ]:
column_names = ['Year', 'Country', 'Happiness', 'Economy', 'Health', 'Freedom', 'Generosity', 'Corruption']

'''
Year: Founding year moduloed and adjusted
Country: Country
Happiness: Sentiment Analysis
Economy: Stock Price
Health: Profit Margin
Freedom: DividendYield
Corruption: MarketCapitalization

Generosity: How much to buy (or not)
'''

In [26]:
api_key = "2RFQ9MPXDZ19H8ZB"
# https://www.alphavantage.co/documentation/

import requests

# replace the "demo" apikey below with your own key from https://www.alphavantage.co/support/#api-key
url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

url = f'https://www.alphavantage.co/query?function=OVERVIEW&symbol=AAPL&apikey={api_key}'
r = requests.get(url)
data = r.json()

print(data)

print("Country:\t", data["Country"])

{'Symbol': 'AAPL', 'AssetType': 'Common Stock', 'Name': 'Apple Inc', 'Description': "Apple Inc. is an American multinational technology company that specializes in consumer electronics, computer software, and online services. Apple is the world's largest technology company by revenue (totalling $274.5 billion in 2020) and, since January 2021, the world's most valuable company. As of 2021, Apple is the world's fourth-largest PC vendor by unit sales, and fourth-largest smartphone manufacturer. It is one of the Big Five American information technology companies, along with Amazon, Google, Microsoft, and Facebook.", 'CIK': '320193', 'Exchange': 'NASDAQ', 'Currency': 'USD', 'Country': 'USA', 'Sector': 'TECHNOLOGY', 'Industry': 'ELECTRONIC COMPUTERS', 'Address': 'ONE INFINITE LOOP, CUPERTINO, CA, US', 'FiscalYearEnd': 'September', 'LatestQuarter': '2022-12-31', 'MarketCapitalization': '2420592607000', 'EBITDA': '125287997000', 'PERatio': '25.63', 'PEGRatio': '2.75', 'BookValue': '3.581', 'Di

In [29]:
def get_year(company_founding_year):
    return (company_founding_year%8)+2016

In [30]:
for i in range(1990, 2024):
    print(f"Company: {i}\tYear: {get_year(i)}")

Company: 1990	Year: 2022
Company: 1991	Year: 2023
Company: 1992	Year: 2016
Company: 1993	Year: 2017
Company: 1994	Year: 2018
Company: 1995	Year: 2019
Company: 1996	Year: 2020
Company: 1997	Year: 2021
Company: 1998	Year: 2022
Company: 1999	Year: 2023
Company: 2000	Year: 2016
Company: 2001	Year: 2017
Company: 2002	Year: 2018
Company: 2003	Year: 2019
Company: 2004	Year: 2020
Company: 2005	Year: 2021
Company: 2006	Year: 2022
Company: 2007	Year: 2023
Company: 2008	Year: 2016
Company: 2009	Year: 2017
Company: 2010	Year: 2018
Company: 2011	Year: 2019
Company: 2012	Year: 2020
Company: 2013	Year: 2021
Company: 2014	Year: 2022
Company: 2015	Year: 2023
Company: 2016	Year: 2016
Company: 2017	Year: 2017
Company: 2018	Year: 2018
Company: 2019	Year: 2019
Company: 2020	Year: 2020
Company: 2021	Year: 2021
Company: 2022	Year: 2022
Company: 2023	Year: 2023


In [33]:
!pip install urllib2
!pip install BeautifulSoup

import urllib2
from BeautifulSoup import BeautifulSoup

LINK = "http://investing.businessweek.com/research/stocks/private/snapshot.asp?privcapId=1716794"
opener = urllib2.build_opener()
opener.addheaders = [('User-agent', 'Google Chrome')]

soup = BeautifulSoup(opener.open(LINK))

location = soup.find('div', {'itemprop': 'address'}).findAll('p')[-1].text
founded = soup.find('span', {'itemprop': "foundingDate"}).text

print (location, founded)

ERROR: Could not find a version that satisfies the requirement urllib2 (from versions: none)
ERROR: No matching distribution found for urllib2

[notice] A new release of pip available: 22.2.2 -> 23.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [7 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 34, in <module>
    File "C:\Users\Sixtium\AppData\Local\Temp\pip-install-cm8xo508\beautifulsoup_b55a688173fd42568c6a0012c85f1f98\setup.py", line 3
      "You're trying to run a very old release of Beautiful Soup under Python 3. This will not work."<>"Please use Beautiful Soup 4, available through the pip package 'beautifulsoup4'."
                                                                                                     ^^
  SyntaxError: invalid syntax
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: metadata-generation-failed

Encountered error while generating package metadata.

See above for output.

note: This is an issue with the package mentioned above, no

ModuleNotFoundError: No module named 'urllib2'